In [49]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
# Load the dataset and display the first 5 rows
df = pd.read_csv('./Loan_default.csv')
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


The target feature for this dataset is 'Default' as it's the column with 0's and 1's as values due to this being a binary classification task.

# 1. Perform Exploratory Data Analysis (EDA)

### Number of rows and columns

In [51]:
rows, cols = df.shape
print(f'Rows: {rows}')
print(f'Columns: {cols}')

Rows: 255347
Columns: 18


### Data Types

In [52]:
print('Data Types: ')
df.dtypes

Data Types: 


LoanID             object
Age                 int64
Income              int64
LoanAmount          int64
CreditScore         int64
MonthsEmployed      int64
NumCreditLines      int64
InterestRate      float64
LoanTerm            int64
DTIRatio          float64
Education          object
EmploymentType     object
MaritalStatus      object
HasMortgage        object
HasDependents      object
LoanPurpose        object
HasCoSigner        object
Default             int64
dtype: object

There is a mostly even mix of numerical and categorical features. We plan to use one-hot encoding on the categorical features and scale the numerical features.

### Statistics for numerical columns

In [53]:
# Generate summary statistics and round them
summary_stats = df.describe().round(2)

# Convert the summary statistics into a DataFrame
summary_stats_df = pd.DataFrame(summary_stats)

summary_stats_df.drop(columns='Default')

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio
count,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00
mean,43.50,82499.30,127578.87,574.26,59.54,2.50,13.49,36.03,0.50
std,14.99,38963.01,70840.71,158.90,34.64,1.12,6.64,16.97,0.23
min,18.00,15000.00,5000.00,300.00,0.00,1.00,2.00,12.00,0.10
25%,31.00,48825.50,66156.00,437.00,30.00,2.00,7.77,24.00,0.30
50%,43.00,82466.00,127556.00,574.00,60.00,2.00,13.46,36.00,0.50
75%,56.00,116219.00,188985.00,712.00,90.00,3.00,19.25,48.00,0.70
max,69.00,149999.00,249999.00,849.00,119.00,4.00,25.00,60.00,0.90


### Statistics for categorical columns

In [54]:
# Create an empty list to store the analysis for each categorical column
categorical_analysis = []

# Loop through each categorical column and gather information
categorical_columns = df.select_dtypes(include=['object'])

for col in categorical_columns:
    # Get unique value count
    unique_values = df[col].nunique()
    
    # Get the mode (most frequent value)
    mode = df[col].mode()[0]
    mode_count = df[col].value_counts().iloc[0]
    mode_pct = (mode_count / len(df)) * 100

    # Get the second mode (second most frequent value if it exists)
    if len(df[col].value_counts()) > 1:
        second_mode = df[col].value_counts().index[1]
        second_mode_count = df[col].value_counts().iloc[1]
        second_mode_pct = (second_mode_count / len(df)) * 100
    else:
        second_mode = None
        second_mode_count = 0
        second_mode_pct = 0

    # Append the results to the list
    categorical_analysis.append({
        'Feature': col,
        'Unique Values': unique_values,
        'Mode': mode,
        'Mode Count': mode_count,
        'Mode %': mode_pct,
        'Second Mode': second_mode,
        'Second Mode Count': second_mode_count,
        'Second Mode %': second_mode_pct
    })

# Convert the list of dictionaries into a DataFrame
categorical_analysis_df = pd.DataFrame(categorical_analysis)

# Display the resulting DataFrame without index
categorical_analysis_df.style.hide()

KeyboardInterrupt: 

In [15]:
def detect_outliers_iqr(df, column):
    """
    Detect outliers in a column using the IQR method.
    
    Parameters:
    df (DataFrame): The DataFrame containing the data.
    column (str): The column to check for outliers.
    
    Returns:
    DataFrame: A DataFrame containing the outliers for that column.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    lower_outliers = df[df[column] < lower_bound]
    upper_outliers = df[df[column] > upper_bound]
    # outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outliers = pd.concat([lower_outliers, upper_outliers], axis=0)
    
    return outliers, lower_outliers, upper_outliers, lower_bound, upper_bound


# Step 1: Select all numerical columns from the DataFrame
numerical_columns = df.select_dtypes(include=['float64', 'int64']).drop(columns='Default').columns

# Step 2: Detect outliers for each numerical column
outliers_dict = {}
for col in numerical_columns:
    outliers, *_ = detect_outliers_iqr(df, col)
    if not outliers.empty:  # Only store columns with outliers
        outliers_dict[col] = outliers

# Step 3: Display the outliers for each numerical column
if outliers_dict:
    print("Using IQR Rule we found outliers in:")
    for col, outliers in outliers_dict.items():
        print(f"  '{col}': {len(outliers)} entries")
else:
    print("No outliers detected in any numerical columns.")


No outliers detected in any numerical columns.


In [16]:
def detect_outliers_n_sigma_rule(df, column, n=3):
    """
    Detect outliers using the n sigma rule (Z-score method).
    
    Parameters:
    df (DataFrame): The DataFrame containing the data.
    column (str): The column to check for outliers.
    threshold (float): The Z-score threshold for defining outliers (default is 3).
    
    Returns:
    DataFrame: A DataFrame containing the outliers.
    """
    mean_col = np.mean(df[column])
    std_col = np.std(df[column])
    
    # Calculate the Z-scores
    z_scores = (df[column] - mean_col) / std_col
    
    # Find outliers based on the threshold
    outliers = df[np.abs(z_scores) > n]
    
    return outliers

# Apply 3 sigma rule method for outlier detection in all numerical columns
outliers_zscore_dict = {}
for col in numerical_columns:
    outliers = detect_outliers_n_sigma_rule(df, col, n=3)
    if not outliers.empty:
        outliers_zscore_dict[col] = outliers

# Display the Z-Score based outliers
if outliers_zscore_dict:
    for col, outliers in outliers_zscore_dict.items():
        print(f"Outliers detected in '{col}' using the 3 Sigma Rule: {len(outliers)} entries.")
else:
    print("No outliers detected using Z-Score.")

No outliers detected using Z-Score.


In [34]:
df.dtypes

LoanID             object
Age                 int64
Income              int64
LoanAmount          int64
CreditScore         int64
MonthsEmployed      int64
NumCreditLines      int64
InterestRate      float64
LoanTerm            int64
DTIRatio          float64
Education          object
EmploymentType     object
MaritalStatus      object
HasMortgage        object
HasDependents      object
LoanPurpose        object
HasCoSigner        object
Default             int64
dtype: object

In [55]:
# use one-hot encoding for categorical features
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# get the categorical and numerical features
categorical_features = df.select_dtypes(include='object').columns.drop('LoanID')
numerical_features = df.select_dtypes(include='number').columns.drop('Default')

one_hot = OneHotEncoder(sparse_output=True, drop='first')
one_hot_encoded = one_hot.fit_transform(df[categorical_features])
encoded_columns = one_hot.get_feature_names_out(categorical_features)

# make dataframe from encoded sparse matrix
df_one_hot = pd.DataFrame.sparse.from_spmatrix(one_hot_encoded, columns=encoded_columns)

# concatenate with the original DataFrame after dropping original categorical features
df_encoded = pd.concat([df.drop(columns=categorical_features).reset_index(drop=True), df_one_hot.reset_index(drop=True)], axis=1)

# Initialize StandardScaler
scaler = StandardScaler()

df_encoded[numerical_features] = scaler.fit_transform(df[numerical_features])

df_encoded

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,...,EmploymentType_Unemployed,MaritalStatus_Married,MaritalStatus_Single,HasMortgage_Yes,HasDependents_Yes,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_Yes
0,I38PQUQS96,0.833990,0.089693,-1.086833,-0.341492,0.590533,1.341937,0.261771,-0.001526,-0.260753,...,0,0,0,1.0,1.0,0,0,0,1.0,1.0
1,HPSK72WA7R,1.701221,-0.823021,-0.044309,-0.731666,-1.285731,-1.343791,-1.308350,1.412793,0.778585,...,0,1.0,0,0,0,0,0,0,1.0,1.0
2,C1OZ6DPJ8Y,0.166888,0.043854,0.022715,-0.775718,-0.968209,0.446694,1.156831,-0.708685,-0.823728,...,1.0,0,0,1.0,1.0,0,0,0,0,0
3,V2KKSFM3UN,-0.767053,-1.303452,-1.168538,1.061875,-1.718715,0.446694,-0.967805,-0.708685,-1.170174,...,0,1.0,0,0,0,1.0,0,0,0,0
4,EY08JDHTZP,1.100830,-1.592855,-1.671921,0.369631,-1.487790,1.341937,-1.052188,0.705634,0.995114,...,1.0,0,0,0,1.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,-1.634285,-1.142632,1.173101,-0.209337,1.427636,1.341937,0.093006,-1.415845,1.514783,...,0,1.0,0,0,0,0,0,0,1.0,0
255343,98R4KDHNND,-0.767053,-0.783984,0.879724,-0.398130,-1.314597,-0.448549,-0.292744,-0.708685,-1.256785,...,0,0,0,0,0,0,0,1.0,0,0
255344,XQK1UUUNGP,0.833990,0.059562,1.139391,0.143078,0.301877,0.446694,-1.236022,1.412793,-0.000918,...,0,1.0,0,1.0,1.0,0,0,0,0,1.0
255345,JAO28CPL4H,-0.099952,0.066979,-0.945840,1.477221,-0.564091,-1.343791,1.116146,0.705634,-0.260753,...,0,0,1.0,1.0,1.0,0,0,0,1.0,0


In [56]:
df_encoded.to_csv('./preprocessed_data.csv')